# Metagene plots
Now, let's draw **metagene plots**: (using seprate conda env and ipython kernel)

Using my `guitar` envirnment. 

In [1]:
%reload_ext rpy2.ipython

In [2]:
%%R
suppressMessages(suppressWarnings(library (Guitar)))

In [7]:
%%R 
txdb <- makeTxDbFromGFF('/rumi/shams/abe/genomes/hg38/gencode.v34/gencode.v34.annotation.gtf',organism='Homo sapiens')

R[write to console]: Import genomic features from the file as a GRanges object ... 
R[write to console]: OK

R[write to console]: Prepare the 'metadata' data frame ... 
R[write to console]: OK

R[write to console]: Make the TxDb object ... 
R[write to console]: OK



In [8]:
%%R 
GuitarPlot(txTxdb = txdb,stBedFiles = list("radar/N-EGF15vs0-result-sig.bed"),miscOutFilePrefix = "Guitar")

[1] "20210916153854"
[1] "There are 228048 transcripts of 60669 genes in the genome."
[1] "total 228048 transcripts extracted ..."
[1] "total 97941 transcripts left after ambiguity filter ..."
[1] "total 97941 transcripts left after check chromosome validity ..."
[1] "total 10939 mRNAs left after component length filter ..."
[1] "total 60598 ncRNAs left after ncRNA length filter ..."
[1] "generate components for all tx"
[1] "generate components for mRNA"
[1] "generate components for lncRNA"
[1] "generate chiped transcriptome"
[1] "generate coverage checking ranges for tx"
[1] "generate coverage checking ranges for mrna"
[1] "generate coverage checking ranges for ncrna"
[1] "20210916154427"
[1] "import BED file radar/N-EGF15vs0-result-sig.bed"
[1] "sample 10 points for Group1"
[1] "start figure plotting for tx ..."
[1] "start figure plotting for mrna ..."
[1] "start figure plotting for ncrna ..."


In [9]:
mv -v Guitar_* plots/

renamed 'Guitar_mrna_test.pdf' -> 'plots/Guitar_mrna_test.pdf'
renamed 'Guitar_ncrna_test.pdf' -> 'plots/Guitar_ncrna_test.pdf'
renamed 'Guitar_tx_test.pdf' -> 'plots/Guitar_tx_test.pdf'


In [1]:
!for f in plots/Guitar*.pdf; do bash ~/Workflows/my_scripts/pdf2png.sh $f; done

plots/Guitar_mrna_test.pdf > plots/Guitar_mrna_test.png
done!
plots/Guitar_ncrna_test.pdf > plots/Guitar_ncrna_test.png
done!
plots/Guitar_tx_test.pdf > plots/Guitar_tx_test.png
done!


# iPAGE
https://github.com/goodarzilab/PAGE

In [20]:
%%bash
jwd='radar'
exp_file='N-EGF15vs0.txt'
ipage_cm='~/GitHub/Abe/my_scripts/ipage_human_ensembl.sh'

cwd=`pwd`
echo $cwd

cd $jwd
nohup ls $exp_file | parallel -j18 -k bash $ipage_cm {} &> ipage.out &
cd $cwd

/rumi/shams/abe/People/Marcelo


In [2]:
!for f in radar/N-EGF15vs0/*pdf; do bash ~/Workflows/my_scripts/pdf2png.sh $f; done

radar/N-EGF15vs0/human_ensembl.all.pdf > radar/N-EGF15vs0/human_ensembl.all.png
done!
radar/N-EGF15vs0/human_ensembl_encode_tf.all.pdf > radar/N-EGF15vs0/human_ensembl_encode_tf.all.png
done!
radar/N-EGF15vs0/human_ensembl.L.pdf > radar/N-EGF15vs0/human_ensembl.L.png
done!
radar/N-EGF15vs0/human_ensembl_msigdb_c1.all.pdf > radar/N-EGF15vs0/human_ensembl_msigdb_c1.all.png
done!
radar/N-EGF15vs0/human_ensembl_msigdb_c1.L.pdf > radar/N-EGF15vs0/human_ensembl_msigdb_c1.L.png
done!
radar/N-EGF15vs0/human_ensembl_msigdb_c2.all.pdf > radar/N-EGF15vs0/human_ensembl_msigdb_c2.all.png
done!
radar/N-EGF15vs0/human_ensembl_msigdb_c2.L.pdf > radar/N-EGF15vs0/human_ensembl_msigdb_c2.L.png
done!
radar/N-EGF15vs0/human_ensembl_msigdb_c2.R.pdf > radar/N-EGF15vs0/human_ensembl_msigdb_c2.R.png
done!
radar/N-EGF15vs0/human_ensembl_msigdb_c3.all.pdf > radar/N-EGF15vs0/human_ensembl_msigdb_c3.all.png
done!
radar/N-EGF15vs0/human_ensembl_msigdb_c3.L.pdf > radar/N-EGF15vs0/human_ensembl_msigdb_c3.L.png
done!


**Write README.md draft**

Write HTML codes which link all plots into a README.md format to prepare GitHub friendly report


In [5]:
ls 

bam/                       N-EGF15vs0-enrichment.md  radar-diff-peaks.ipynb
downstream-analysis.ipynb  plots/                    ruddle_paths.txt
fastq/                     preprocessing-QC/         star1.out
logs/                      radar/                    star.out


In [4]:
%%bash 
readme='N-EGF15vs0-enrichment.md' 
touch $readme
for f in radar/N-EGF15vs0/*.png; do 
    b=`basename $f` t=${b/.png/}
    echo '#### '$t >> $readme
    echo -e "&lt;img src=\""$f"\" title=\""$t"\" style=\"width:1000px\">\n" >> $readme 
done

# FIRE 
https://github.com/goodarzilab/FIRE
### Motif analysis 

Using my `cgat` envirnment. 

In [13]:
%%bash 
jwd=radar
BED=N-EGF15vs0-result-all.bed

cwd=`pwd`
echo $cwd

MOTIF=/rumi/shams/abe/GitHub/imRIP/motifs.txt
FA=${BED/.bed/.fa}
FIRE_FA=${FA/.fa/_fire.fa}
FIRE_EXP=${FA/.fa/_fire.txt}

cd $jwd
echo "_____________________________ `basename $BED` ____________________________________________"

echo "step 1: extract mRNA sequences"
cat $BED | sort -k1,1 -k2,2n $BED | cgat bed2bed --method=merge --merge-by-name |  awk '! /#/' | bedtools getfasta -name -s -fi /rumi/shams/genomes/hg38/hg38.fa -bed - -split -fo $FA
echo "--- DONE! ---"

echo "step 2: prepare inputs for FIRE"
# perl $TEISERDIR/prep_seqs_for_teiser_run.pl peak.fa peaks
/rumi/shams/abe/anaconda3/envs/cgat/bin/python $TEISERDIR/prep_fasta_for_fire_run.py $FA

echo "--- DONE! ---"

cd $cwd

/rumi/shams/abe/People/Marcelo
_____________________________ N-EGF15vs0-result-all.bed ____________________________________________
step 1: extract mRNA sequences
--- DONE! ---
step 2: prepare inputs for FIRE
--- DONE! ---


In [14]:
%%bash
jwd=radar
BED=N-EGF15vs0-result-sig.bed

cwd=`pwd`
echo $cwd

MOTIF=/rumi/shams/abe/GitHub/imRIP/motifs.txt
FA=${BED/.bed/.fa}
FIRE_FA=${FA/.fa/_fire.fa}
FIRE_EXP=${FA/.fa/_fire.txt}


cd $jwd

echo "step 3: run FIRE for known m6A motifs (non-discovery mode)"
cat $MOTIF
perl $FIREDIR/fire.pl --expfile=$FIRE_EXP --exptype=discrete --fastafile_rna=$FIRE_FA \
--nodups=1 --dodna=0 --dodnarna=0 --species=human --doskipdiscovery=1 \
--motiffile_rna=$MOTIF --oribiasonly=0 > non-discovery_FIRE.log
mv -v ${FIRE_EXP}_FIRE/ non-discovery_FIRE
echo "--- DONE! ---"

echo "step 4: run FIRE discovery mode"
perl $FIREDIR/fire.pl --expfile=$FIRE_EXP --exptype=discrete --fastafile_rna=$FIRE_FA \
--nodups=1 --dodna=0 --dodnarna=0 --species=human --oribiasonly=0 > discovery_FIRE.log 
mv -v ${FIRE_EXP}_FIRE/ discovery_FIRE
echo "--- DONE! ---"

cd $cwd

/rumi/shams/abe/People/Marcelo
step 3: run FIRE for known m6A motifs (non-discovery mode)
'N-EGF15vs0-result-sig_fire.txt_FIRE/' -> 'non-discovery_FIRE'
--- DONE! ---
step 4: run FIRE discovery mode
'N-EGF15vs0-result-sig_fire.txt_FIRE/' -> 'discovery_FIRE'
--- DONE! ---


No motifs.
Can't use 'defined(@array)' (Maybe you should just omit the defined()?) at /flash/bin/FIRE-1.1/SCRIPTS/draw_position_histogram.pl line 134.
Table.pm: cannot open file "N-EGF15vs0-result-sig_fire.txt_FIRE/RNA/N-EGF15vs0-result-sig_fire.txt.clusters" ..
Table.pm: cannot open file "N-EGF15vs0-result-sig_fire.txt_FIRE/RNA/N-EGF15vs0-result-sig_fire.txt.mimatrix" ..
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
No motifs in @MOTIFS ...
N-EGF15vs0-result-sig_fire.txt_FIRE/RNA/N-EGF15vs0-result-sig_fire.txt.fullmimatrix does not exists. at /flash/bin/FIRE-1.1/SCRIPTS/Sets.pm line 165.
N-EGF15vs0-result-sig_fire.txt_FIRE/RNA/N-EGF15vs0-result-sig_fire.txt.fullmimatrix does not exists. at /flash/bin/FIRE-1.1/SCRIPTS/Sets.pm line 165.


In [15]:
ls 

bam/                       logs/              radar/                  star1.out
downstream-analysis.ipynb  plots/             radar-diff-peaks.ipynb  star.out
fastq/                     preprocessing-QC/  ruddle_paths.txt


### DRACH & RGAC motifs

<img src=fire/known_motifs_FIRE.png style="width:1100px">

## Discover over/under represented motifs 

<img src=fire/discovery_FIRE.summary.png style="width:1100px">